In [1]:
root_dir = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner'
dataset_dir = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/data/dataset'
ultralytics_from = '/opt/homebrew/anaconda3/envs/torch_env/lib/python3.10/site-packages/ultralytics/'
ultralytics_to = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/modules/ultralytics'

In [2]:
import os, sys, shutil, yaml
from importlib import reload
from glob import glob
from zipfile import ZipFile

import pandas as pd

import ultralytics
from ultralytics import YOLO
# os.symlink(ultralytics_from, ultralytics_to)

In [2]:
augment_py = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/modules/ultralytics/data/augment.py'
# augment_py = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/modules/ultralytics/data/augment(copy).py'

def custom_albumentations(p_Blur=0.2, p_Defocus=0.2, p_MotionBlur=0.5, p_MedianBlur=0.05, p_ZoomBlur=0.05,
                          p_FancyPCA=1.0, p_ToGray=1.0, p_colors=0.5,
                          p_CLAHE=0.5, p_Emboss=0.5, p_ISONoise=0.3,
                          p_RandomBrightnessContrast=0.00, p_RandomGamma=0.00, p_ImageCompression=[75, 0.0]):

    # read augment.py
    with open(augment_py, 'r') as f:
        data = f.readlines() # class Albumentations - define hyp

    # start_idx, end_idx가 불확실하기 때문에, 수정 및 실행 전에 원본 파일 백업하고 작업하시는 것을 추천드립니다.
    aug_start_idx = data.index('            T = [\n') + 1
    aug_end_idx = data.index("            self.transform = A.Compose(T, bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))\n")


    del data[aug_start_idx : aug_end_idx+1]
    modified = ''
    modified += f'                # Blurs\n'
    modified += f'                A.Blur(blur_limit=(3, 10), p={p_Blur}),\n'
    modified += f'                A.Defocus(radius=(3, 10), alias_blur=(0.1, 0.5), always_apply=False, p={p_Defocus}),\n'
    modified += f'                A.MotionBlur(blur_limit=21, allow_shifted=True, always_apply=False, p={p_MotionBlur}),\n'
    modified += f'                A.MedianBlur(p={p_MedianBlur}),\n'
    modified += f'                A.ZoomBlur(max_factor=1.2, step_factor=(0.01, 0.01), always_apply=False, p={p_ZoomBlur}),\n'
    modified += f'                # colors\n'
    modified += f'                A.OneOf([\n'
    modified += f'                    A.FancyPCA(alpha=0.1, always_apply=False, p={p_FancyPCA}),\n'
    modified += f'                    A.ToGray(p={p_ToGray})],\n'
    modified += f'                    p={p_colors}),\n'
    modified += f'\n'
    modified += f'                # etc\n'
    modified += f'                A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p={p_CLAHE}),\n'
    modified += f'                A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), always_apply=False, p={p_Emboss}),\n'
    modified += f'                A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), always_apply=False, p={p_ISONoise}),\n'
    modified += f'                A.RandomBrightnessContrast(p={p_RandomBrightnessContrast}),\n'
    modified += f'                A.RandomGamma(p={p_RandomGamma}),\n'
    modified += f'                A.ImageCompression(quality_lower={p_ImageCompression[0]}, p={p_ImageCompression[1]})]\n'
    modified += f'            self.transform = A.Compose(T, bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))\n'

    data.insert(aug_start_idx+1, modified)

    # override autment.py
    with open(augment_py, 'w') as f:
        new_contents = ''.join(data)
        f.write(new_contents)


# Albumentations
## Blur
p_Blur=0.3
p_Defocus=0.2
p_MotionBlur=0.5
p_MedianBlur=0.05
p_ZoomBlur=0.05
## Color
p_FancyPCA=1.0
p_ToGray=1.0
p_colors=0.5
## etc
p_CLAHE=0.5
p_Emboss=0.5
p_ISONoise=0.3
p_RandomBrightnessContrast=0.075
p_RandomGamma=0.00
p_ImageCompression=[75, 0.0]

custom_albumentations(p_Blur=p_Blur, p_Defocus=p_Defocus, p_MotionBlur=p_MotionBlur,
                      p_MedianBlur=p_MedianBlur, p_ZoomBlur=p_ZoomBlur, p_FancyPCA=p_FancyPCA,
                      p_ToGray=p_ToGray, p_colors=p_colors, p_CLAHE=p_CLAHE, p_Emboss=p_Emboss,
                      p_ISONoise=p_ISONoise, p_RandomBrightnessContrast=p_RandomBrightnessContrast,
                      p_RandomGamma=p_RandomGamma, p_ImageCompression=p_ImageCompression)

In [13]:
# def custom_albumentations(p_Blur=0.2, p_Defocus=0.2, p_MotionBlur=0.5, p_MedianBlur=0.05, p_ZoomBlur=0.05, # Blur
#                           p_FancyPCA=1.0, p_ToGray=1.0, p_colors=0.5, # Color
#                           p_CLAHE=0.5, p_Emboss=0.5, p_ISONoise=0.3,
#                           p_RandomBrightnessContrast=0.00, p_RandomGamma=0.00, p_ImageCompression=[75, 0.0]):
#     augment_py = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/modules/ultralytics/data/augment.py'
#     # augment_py = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/modules/ultralytics/data/augment(copy).py'
#
#     # read augment.py
#     with open(augment_py, 'r') as f:
#         data = f.readlines() # class Albumentations - define hyp
#
#     aug_start_idx = data.index('            T = [\n') + 1
#     aug_end_idx = data.index("            self.transform = A.Compose(T, bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))\n")
#
#
#     del data[aug_start_idx : aug_end_idx]
#     modified = ''
#     modified += f'                # Blurs\n'
#     modified += f'                A.Blur(blur_limit=(3, 10), p={p_Blur}),\n'
#     modified += f'                A.Defocus(radius=(3, 10), alias_blur=(0.1, 0.5), always_apply=False, p={p_Defocus}),\n'
#     modified += f'                A.MotionBlur(blur_limit=21, allow_shifted=True, always_apply=False, p={p_MotionBlur}),\n'
#     modified += f'                A.MedianBlur(p={p_MedianBlur}),\n'
#     modified += f'                A.ZoomBlur(max_factor=1.2, step_factor=(0.01, 0.01), always_apply=False, p={p_ZoomBlur}),\n'
#     modified += f'                # colors\n'
#     modified += f'                A.OneOf([\n'
#     modified += f'                    A.FancyPCA(alpha=0.1, always_apply=False, p={p_FancyPCA}),\n'
#     modified += f'                    A.ToGray(p={p_ToGray})],\n'
#     modified += f'                    p={p_colors}),\n'
#     modified += f'\n'
#     modified += f'                # etc\n'
#     modified += f'                A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p={p_CLAHE}),\n'
#     modified += f'                A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), always_apply=False, p={p_Emboss}),\n'
#     modified += f'                A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), always_apply=False, p={p_ISONoise}),\n'
#     modified += f'                A.RandomBrightnessContrast(p={p_RandomBrightnessContrast}),\n'
#     modified += f'                A.RandomGamma(p={p_RandomGamma}),\n'
#     modified += f'                A.ImageCompression(quality_lower={p_ImageCompression[0]}, p={p_ImageCompression[1]})]\n'
#
#     data.insert(aug_start_idx, modified)
#
#     # override autment.py
#     with open(augment_py, 'w') as f:
#         new_contents = ''.join(data)
#         f.write(new_contents)
#
#
# # Albumentations
# ## Blur
# ## Blur
# p_Blur=0.3
# p_Defocus=0.2
# p_MotionBlur=0.5
# p_MedianBlur=0.05
# p_ZoomBlur=0.05
# ## Color
# p_FancyPCA=1.0
# p_ToGray=1.0
# p_colors=0.5
# ## etc
# p_CLAHE=0.5
# p_Emboss=0.5
# p_ISONoise=0.3
# p_RandomBrightnessContrast=0.075
# p_RandomGamma=0.00
# p_ImageCompression=[75, 0.0]
#
# custom_albumentations(p_Blur=p_Blur, p_Defocus=p_Defocus, p_MotionBlur=p_MotionBlur,
#                       p_MedianBlur=p_MedianBlur, p_ZoomBlur=p_ZoomBlur, p_FancyPCA=p_FancyPCA,
#                       p_ToGray=p_ToGray, p_colors=p_colors, p_CLAHE=p_CLAHE, p_Emboss=p_Emboss,
#                       p_ISONoise=p_ISONoise, p_RandomBrightnessContrast=p_RandomBrightnessContrast,
#                       p_RandomGamma=p_RandomGamma, p_ImageCompression=p_ImageCompression)

# 학습

In [5]:
data_yaml = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/data/dataset3/data.yaml'
save_dir = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/results/yolov8n_'
# pretrained_weight = '/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/results/yolov8n_albu(off)/weights/best.pt'

# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
# model = YOLO(pretrained_weight)  # 학습한 weight


model.train(data=data_yaml, epochs=300, batch=64, patience=5,
            name= save_dir + 'ds3_20th_exp',
            verbose=True, cache=True,
            resume=False,
            # 아래에 augmentation argumenet 추가
            # hsv_h = 0.0, hsv_s = 0.0, hsv_v = 0.0
            # mixup = 0.3,
            # perspective = 0.001,
            )  # train the model

# # 마지막 기준으로 1에폭 돌리기
# model.train(data=data_yaml, epochs=1, batch=64,
#             name=save_dir + 'tmp',
#             verbose=True, workers=10,
#             )

# augmentation default settings
'''
lr0: 0.01  # (float) initial learning rate (i.e. SGD=1E-2, Adam=1E-3)
lrf: 0.01  # (float) final learning rate (lr0 * lrf)
momentum: 0.937  # (float) SGD momentum/Adam beta1
weight_decay: 0.0005  # (float) optimizer weight decay 5e-4
warmup_epochs: 3.0  # (float) warmup epochs (fractions ok)
warmup_momentum: 0.8  # (float) warmup initial momentum
warmup_bias_lr: 0.1  # (float) warmup initial bias lr
box: 7.5  # (float) box loss gain
cls: 0.5  # (float) cls loss gain (scale with pixels)
dfl: 1.5  # (float) dfl loss gain
pose: 12.0  # (float) pose loss gain
kobj: 1.0  # (float) keypoint obj loss gain
label_smoothing: 0.0  # (float) label smoothing (fraction)
nbs: 64  # (int) nominal batch size
hsv_h: 0.015  # (float) image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # (float) image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # (float) image HSV-Value augmentation (fraction)
degrees: 0.0  # (float) image rotation (+/- deg)
translate: 0.1  # (float) image translation (+/- fraction)
scale: 0.5  # (float) image scale (+/- gain)
shear: 0.0  # (float) image shear (+/- deg)
perspective: 0.0  # (float) image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # (float) image flip up-down (probability)
fliplr: 0.5  # (float) image flip left-right (probability)
mosaic: 1.0  # (float) image mosaic (probability)
mixup: 0.0  # (float) image mixup (probability)
copy_paste: 0.0  # (float) segment copy-paste (probability)
'''

New https://pypi.org/project/ultralytics/8.0.149 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.147 🚀 Python-3.10.11 torch-2.0.1 CPU (Apple M1 Max)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/data/dataset3/data.yaml, epochs=300, patience=5, batch=64, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=/Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/results/yolov8n_ds3_20th_exp, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

train: Scanning /Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/data/dataset3/train/labels... 6696 images, 2 backgrounds, 0 corrupt: 100%|██████████| 6696/6696 [00:01<00:00, 3967.24it/s]
train: New cache created: /Users/GitHub/Projects/AI_Project/yolov8-robot-cleaner/data/dataset3/train/labels.cache
train: Caching images (4.3GB True): 100%|██████████| 6696/6696 [00:01<00:00, 3372.04it/s]
albumentations: Blur(p=0.3, blur_limit=(3, 10)), Defocus(p=0.2, radius=(3, 10), alias_blur=(0.1, 0.5)), MotionBlur(p=0.5, blur_limit=(3, 21), allow_shifted=True), MedianBlur(p=0.05, blur_limit=(3, 7)), ZoomBlur(p=0.05, max_factor=(1.0, 1.2), step_factor=(0.01, 0.01)), OneOf([
  FancyPCA(p=1.0, alpha=0.1),
  ToGray(p=1.0),
], p=0.5), CLAHE(p=0.5, clip_limit=(1, 4.0), tile_grid_size=(8, 8)), Emboss(p=0.5, alpha=(0.2, 0.5), strength=(0.2, 0.7)), ISONoise(p=0.3, intensity=(0.1, 0.5), color_shift=(0.01, 0.05)), RandomBrightnessContrast(p=0.075, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2)

KeyboardInterrupt: 